In [1]:
import os

In [2]:
%pwd

'/home/cdot/PycharmProjects/ChickeDecease/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/cdot/PycharmProjects/ChickeDecease'

### Initalizing the entity

In [29]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: str(Path)
    tensorboard_root_log_dir: str(Path)
    checkpoint_model_filepath: str(Path)

### Update Config manager from src

In [30]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = str(os.path.dirname(config.checkpoint_model_filepath))
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config




### Update the Components

In [31]:

import os
import urllib.request as req
from zipfile import ZipFile
import time
import tensorflow as tf

In [34]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


### Update pipeline

In [36]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2024-01-01 12:14:35,237: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-01 12:14:35,245: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-01 12:14:35,248: INFO: common: Created directory at: artifacts]
[2024-01-01 12:14:35,252: INFO: common: Created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-01-01 12:14:35,255: INFO: common: Created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]


In [40]:
### Timestamp

import time
ts = time.strftime("%Y-%m-%d-%H-%m-%s")
f"tb_logs_at {ts}"

'tb_logs_at 2024-01-01-13-01-1704097062'